In [75]:
import tensorflow as tf
import tensorflow as tf
import numpy as np

from turtle import up
import pandas as pd
import scipy
from scipy.stats import iqr


def fun1(df, ws=4):

    def windowed_dataset(dataset: tf.data.Dataset, window_size=5, shift=1, stride=1, drop_remainder=False):
        windows = dataset.window(window_size, shift=shift, stride=stride, drop_remainder=True)

        def sub_to_batch(t1, t2=None):
            t1_batches = t1.batch(window_size, drop_remainder=drop_remainder)

            if t2 is not None:
                t2_batches = t2.batch(window_size, drop_remainder=drop_remainder)
                return tf.data.Dataset.zip(t1_batches, t2_batches)
            else:
                return t1_batches

        windows = windows.flat_map(sub_to_batch)
        return windows  #.batch(window_size, drop_remainder=drop_remainder)

    dataset = windowed_dataset(tf.data.Dataset.from_tensor_slices(df.values), ws).batch(batch_size=ws)

    return dataset


def fun2(df, ws=4):

    def windowed_dataset(dataset: tf.data.Dataset, window_size=5, shift=1, stride=1, drop_remainder=False):
        windows = dataset.window(window_size, shift=shift, stride=stride, drop_remainder=True)

        def sub_to_batch(t1, t2=None):
            t1_batches = t1.batch(window_size, drop_remainder=drop_remainder)

            if t2 is not None:
                t2_batches = t2.batch(window_size, drop_remainder=drop_remainder)
                return tf.data.Dataset.zip(t1_batches, t2_batches)
            else:
                return t1_batches

        windows = windows.flat_map(sub_to_batch)
        return windows.batch(window_size)

    dataset = windowed_dataset(tf.data.Dataset.from_tensor_slices(df.values), ws)  #.batch(batch_size=window_size)

    return dataset

In [76]:
def covert_to_numpy(ds):
    # 将 dataset 转换为 NumPy 数组
    numpy_array = None
    for data in ds.as_numpy_iterator():
        if numpy_array is None:
            numpy_array = np.array(data)
        else:
            numpy_array = np.vstack((numpy_array, data))

    print(numpy_array.shape)
    print(numpy_array)
    return numpy_array

In [77]:
rows = 10
cols = 3

data_np = np.array([[i + 1 for c in range(cols)] for i in range(rows)])

df = pd.DataFrame(data_np, columns=['a', 'b', 'c'])
# print(df.shape)
# print(df)
# 假设我们有一个tf.data.Dataset对象
# 例如，一个包含随机数的 dataset

# print(numpy_array)


In [78]:
from lib2to3.pytree import convert

win_size = 7
ds1 = fun1(df, win_size)
_ = covert_to_numpy(ds1)


(4, 7, 3)
[[[ 1  1  1]
  [ 2  2  2]
  [ 3  3  3]
  [ 4  4  4]
  [ 5  5  5]
  [ 6  6  6]
  [ 7  7  7]]

 [[ 2  2  2]
  [ 3  3  3]
  [ 4  4  4]
  [ 5  5  5]
  [ 6  6  6]
  [ 7  7  7]
  [ 8  8  8]]

 [[ 3  3  3]
  [ 4  4  4]
  [ 5  5  5]
  [ 6  6  6]
  [ 7  7  7]
  [ 8  8  8]
  [ 9  9  9]]

 [[ 4  4  4]
  [ 5  5  5]
  [ 6  6  6]
  [ 7  7  7]
  [ 8  8  8]
  [ 9  9  9]
  [10 10 10]]]


In [79]:
ds2 = fun2(df, win_size)
_ = covert_to_numpy(ds2)


(4, 7, 3)
[[[ 1  1  1]
  [ 2  2  2]
  [ 3  3  3]
  [ 4  4  4]
  [ 5  5  5]
  [ 6  6  6]
  [ 7  7  7]]

 [[ 2  2  2]
  [ 3  3  3]
  [ 4  4  4]
  [ 5  5  5]
  [ 6  6  6]
  [ 7  7  7]
  [ 8  8  8]]

 [[ 3  3  3]
  [ 4  4  4]
  [ 5  5  5]
  [ 6  6  6]
  [ 7  7  7]
  [ 8  8  8]
  [ 9  9  9]]

 [[ 4  4  4]
  [ 5  5  5]
  [ 6  6  6]
  [ 7  7  7]
  [ 8  8  8]
  [ 9  9  9]
  [10 10 10]]]
